# [LB Probing Strategies - [0.890] - 2nd Place](https://www.kaggle.com/cdeotte/lb-probing-strategies-0-890-2nd-place)

## LBプロービングテクニック-Don't Overfit! II-2位の解法
このカーネルでは、Kaggleリーダーボード（LB）をプローブして公開テストデータセットに関する情報を取得するための効率的な戦略について説明します。 LRロジスティック回帰、SVCサポートベクトル分類、およびLDAナイーブベイズは、データを分類する線形超平面を見つける3つの方法です。 「Don't Overfit II」コンテストでは、公開テストデータセットのデータは、トレーニングデータセットの8倍です。 トレーニングデータを分類する超平面を見つける代わりに、公開テストデータセットを分類する超平面を見つけることをお勧めします。 私たちはできる。次の式を仮定しましょう。

$$
\text{target} = \text{Heaviside}(a_0x_0+a_1x_1+...a_{298}x_{298}+a_{299}x_{299}+\text{noise})
$$

ヘヴィサイドステップ関数heaviside(x)は、x<0の場合は値0を、x>0の場合は値1を、x=0の場合は値1/2を返す。  
私たちのタスクは、300の超平面係数a_kを決定することです。 a_kが独立していると仮定すると、次のコードはLBプローブを介してパブリックテストデータセットからそれらを抽出します。

In [2]:
import pandas as pd

In [3]:
var = 33
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')
sub['target'] = test[str(var)]
sub.to_csv('submission'+str(var)+'.csv',index=False) 

testデータの特徴量33の列をそのまま提出ファイルの予測値として提出

In [4]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "http://playagricola.com/Kaggle/sub33.jpg")

a_kの値は、ちょうどLB_SCORE_Kから0.500を引いたものです。 例えば

$$
a_{33} = \text{LB_SCORE}_{33} - 0.500 = 0.671 - 0.500 = 0.171
$$
変数がターゲットと負の相関がある場合（正の相関とは対照的に）、LB_SCORE_Kは0.500未満になります。

In [15]:
Image(url= "http://playagricola.com/Kaggle/sub217.jpg")

$$
a_{217} = \text{LB_SCORE}_{217} - 0.500 = 0.382 - 0.500 = -0.118
$$

とても簡単です！ これを行うことにより、次の3つのトリックを追加することで、20日（100回の送信）でa_kを回復できます。
- 最も重要な100個のa_kのみをプローブします。 abs（CV_SCORE_K-0.5）<0.04の場合、a_k = 0を設定し、プローブしません。
- 精度を高めるには、訓練データと公開データを使用します。 LB_SCORE_Kを（8/9）* LB_SCORE_K +（1/9）* CV_SCORE_Kに置き換えます。
- L1ペナルティを適用します。 abs（LB_SCORE_K-0.5）<0.04の場合、a_k = 0を設定します。


これらの追加のトリックは、LBの過剰適合を防ぐのに役立ちます。 パブリックテストデータセットのみをモデル化してとオーバーフィッティングのリスクをとる代わりに、2225の合計サンプルサイズのトレーニングデータとパブリックテストデータの両方からの情報を使用します。このサンプルサイズでは、AUCが0.54以下のすべての変数が それ自体は役に立たない変数かもしれません。 したがって、abs（AUC-0.5）<0.04のすべての変数をモデルから削除します。ここで、AUC =（8/9）* LB_AUC +（1/9）* CV_AUCは、不要な変数の過剰適合を防ぎます。

ここで説明するように、0.3 <AUC <0.7であるため、線形近似を使用してa_kを見つけていることに注意してください。

## 合成データセット実験¶
この手法を合成データセットで観察してみましょう。 次に、それを実際のデータセットに適用します。 この手法を以下のモデル4に示します。 最初に、他の3つの人気モデルを紹介します。 （最初の2つのモデルはトレーニングデータのみを使用し、次の2つのモデルはパブリックテストデータを使用します）。

In [17]:
import numpy as np, pandas as pd, os
np.random.seed(300)

# GENERATE RANDOM DATA
data = pd.DataFrame(np.zeros((20000,300)))
for i in range(300): 
    data.iloc[:,i] = np.random.normal(0,1,20000)
    
# SET TARGET AS LINEAR COMBINATION OF 50 A'S PLUS NOISE 
important = 35
noise = 3.5
a = np.random.normal(0,1,300)
x = np.random.choice(np.arange(300),300-important,replace=False)
a[x] = 0
data['target'] = data.values.dot(a) + np.random.normal(0,noise,20000)

# MAKE 64% TARGET=1, 36% TARGET=0
data.sort_values('target',inplace=True)
data.iloc[:7200,300] = 0.0
data.iloc[7200:,300] = 1.0

# RANDOMLY SELECT TRAIN, PUBLIC, PRIVATE
train = data.sample(250)
public = data[ ~data.index.isin(train.index) ].sample(1975)
private = data[ ~data.index.isin(train.index) & ~data.index.isin(public.index) ].sample(frac=1) 

# RESET INDICES
train.reset_index(drop=True,inplace=True)
public.reset_index(drop=True,inplace=True)
private.reset_index(drop=True,inplace=True)

## モデル1：トレーニングデータL2-Hyperplane
トレーニングデータのみを使用し、ロジスティック回帰、L2ペナルティ、および層別k分割を使用してモデルを構築する場合、合成データセットでCV 0.772、LB 0.728およびプライベートスコア0.743を達成します。

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score

oof = np.zeros(len(train))
predsPU= np.zeros(len(public))
predsPR= np.zeros(len(private))
rskf = RepeatedStratifiedKFold(n_splits=25, n_repeats=5)
for train_index, test_index in rskf.split(train.iloc[:,:-1], train['target']):
    clf = LogisticRegression(solver='liblinear',penalty='l2',C=1.0,class_weight='balanced')
    clf.fit(train.loc[train_index].iloc[:,:-1],train.loc[train_index]['target'])
    oof[test_index] += clf.predict_proba(train.loc[test_index].iloc[:,:-1])[:,1]
    predsPU += clf.predict_proba(public.iloc[:,:-1])[:,1]
    predsPR += clf.predict_proba(private.iloc[:,:-1])[:,1]
aucTR = round(roc_auc_score(train['target'],oof),5)
aucPU = round(roc_auc_score(public['target'],predsPU),5)
aucPR = round(roc_auc_score(private['target'],predsPR),5)
print('LR Model with L2-penalty: CV =',aucTR,'LB =',aucPU,'Private score =',aucPR)

LR Model with L2-penalty: CV = 0.77491 LB = 0.72832 Private score = 0.74249


## モデル2：トレーニングデータL1-Hyperplane
トレーニングデータのみを使用し、ロジスティック回帰、L1ペナルティ、および層別kフォールドでモデルを構築する場合、合成データセットでCV 0.846、LB 0.841およびプライベートスコア0.837を達成します。

In [19]:
oof = np.zeros(len(train))
predsPU= np.zeros(len(public))
predsPR= np.zeros(len(private))
rskf = RepeatedStratifiedKFold(n_splits=25, n_repeats=5)
for train_index, test_index in rskf.split(train.iloc[:,:-1], train['target']):
    clf = LogisticRegression(solver='liblinear',penalty='l1',C=0.1,class_weight='balanced')
    clf.fit(train.loc[train_index].iloc[:,:-1],train.loc[train_index]['target'])
    oof[test_index] += clf.predict_proba(train.loc[test_index].iloc[:,:-1])[:,1]
    predsPU += clf.predict_proba(public.iloc[:,:-1])[:,1]
    predsPR += clf.predict_proba(private.iloc[:,:-1])[:,1]
aucTR = round(roc_auc_score(train['target'],oof),5)
aucPU = round(roc_auc_score(public['target'],predsPU),5)
aucPR = round(roc_auc_score(private['target'],predsPR),5)
print('LR Model with L1-penalty: CV =',aucTR,'LB =',aucPU,'Private score =',aucPR)

LR Model with L1-penalty: CV = 0.85657 LB = 0.84082 Private score = 0.83736


## モデル3：パブリックデータL2-Hyperplane Search
ランダムLBプロービングを使用して、プライベートスコアを増やします。 トレーニングデータモデルからの最適なL1超平面から始め、ランダムに5度回転させて再送信します。 LBスコアが増加した場合は、新しい超平面を保持します。それ以外の場合は、古い超平面を保持します。 このプロセスを30日間繰り返します（150件の提出）。

このプロセスでは、ランダム検索を使用して、公開テストデータセットで最適なL2-超平面を見つけます。 トレーニングデータセットのみを使用するモデル1および2に勝っていることに注意してください。 トレーニングデータで重要であることが証明されている可変次元を介して超平面を回転させるだけで、この方法を加速できます。 （このアクセラレータは、トレーニングモデルからL1ペナルティを適用するようなものです）

1か月後（150件の送信）、このメソッドは、合成データセットでLBスコア0.885およびプライベートスコア0.859を達成します。